#### Setup

you need to install
```sh
mamba install -c conda-forge mpi4py
```

#  MPI parallelization

* MPI has been a parallelization method since 1991! Many languages are supported (FORTRAN, C, C++, Python, Julia, Java, R...)
* It is used in many large-scale scientific and industrial projects
* The way MPI works is that is executes the same file on multiple machines, but each instance gets a RANK (an integer). 
* The **advantage of MPI** over *multiprocessing* or *concurrent.futures* is that it can run on a large cluster of machines 
* **The disadvantage** is that memory is not shared, it has to be communicated explicity!

* It is up to the code to decide what it should do with that rank: usually it is used to define the part of the problem the worker instance should work on.  There is no "master" instance, all workers can send messages to each other (by rank) or wait for a message (blocking or not).  

Like other parallel systems, *MPI jobs cannot run in a notebook*: they have to be explicit files.

In this example (adapted from https://mpi4py.readthedocs.io/en/stable/tutorial.html), the worker that gets Rank 0 computes some data, and then sends it to the Rank 1 worker, who prints it out.

In [21]:
%%file mpi_demo.py

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

print(f"{rank}: Hello!")

if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    print(f"{rank}: sending {data}")
    comm.Send(data, dest=1, tag=13)
    
elif rank == 1:
    print(f"{rank}: waiting for data from rank 0")
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)
    print(f"{rank}: received {data}")
else:
    print(f"{rank}: I've got nothing to do")

Overwriting mpi_demo.py


####  Launch the jobs using a terminal:
```sh
mpiexec --np 5 python -m mpi4py  mpi_demo.py
```

On a cluster with MPI set up on multiple machines, this would parallelize across 5 machines.  On a single machine, it will run similar to multi-processing in 5 processes

### How would you use this?

Imagine you have a block-parallel problem.  You can for each worker:

1. Set up the problem and initial conditions (identical for all)
1. From the rank, decide which block you are (e.g. geometically)
1. Compute the first iteration from the initial conditions
1. For each *edge* of your block, 
    1. send its contents to the appropriate neighbor
    1. wait to receive the edges from your neighbors
1. do the next iteration

the MPI standard (and mpi4py) provides a lot of helpful functions for this type of work, e.g. to:
* *scatter/gather* data to/from many other workers
* *broadcast* data to all workers

### higher-level: concurrent.futures
mpi2py provides a `MPIPoolExecutor` that works exactly like `ProcessPoolExecutor` or `ThreadPoolExecutor`, which makes it easy to apply to map-style problems
